In [1]:
import os
import json
import xgboost as xgb
from metrics import *
from sklearn.metrics import make_scorer, cohen_kappa_score, balanced_accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer
import numpy as np

In [2]:
# Load features and labels
X = np.load('../../features.npy')
y = np.load('../../labels.npy')

X.shape, y.shape

((8724, 2048), (8724,))

In [3]:
# Count the number of samples in each class
print('Number of samples in each class:')
print(np.unique(y, return_counts=True))

# Define class names
class_names = ["Few", "Many", "None"]
print('Class names:')
print(class_names)

Number of samples in each class:
(array([0, 1, 2]), array([6232,  256, 2236]))
Class names:
['Few', 'Many', 'None']


In [4]:
os.makedirs('logs', exist_ok=True)
os.makedirs('figures', exist_ok=True)

In [5]:
def custom_scorer(y_true, y_pred):
    if y_pred.ndim == 2:
        y_pred_class = np.argmax(y_pred, axis=1)
    else:
        y_pred_class = y_pred

    auc = roc_auc_score(y_true, y_pred, average='macro', multi_class='ovo')
    f1 = f1_score(y_true, y_pred_class, average='macro')
    kappa = cohen_kappa_score(y_true, y_pred_class)
    balanced_acc = balanced_accuracy_score(y_true, y_pred_class)
    
    return 0.3 * auc + 0.3 * f1 + 0.2 * kappa + 0.2 * balanced_acc

custom_score = make_scorer(custom_scorer, greater_is_better=True, needs_proba=True)

/home/samo/dev/liver/.venv/lib/python3.11/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


In [12]:
import cupy as cp

# Convert your data to GPU
X_gpu = cp.array(X)
y_gpu = cp.array(y)


In [7]:
xgb_clf = xgb.XGBClassifier(objective='multi:softprob', device= "cuda", num_class=3, random_state=42)

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'min_child_weight': [1, 3, 5]
}

In [13]:
random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_grid,
    n_iter=100,
    cv=5,
    verbose=3,
    random_state=42,
    # n_jobs=6,
    scoring=custom_score
)

random_search.fit(X_gpu.get(), y_gpu.get())

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=1, n_estimators=500, subsample=0.8;, score=0.924 total time=   8.1s
[CV 2/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=1, n_estimators=500, subsample=0.8;, score=0.935 total time=   8.4s
[CV 3/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=1, n_estimators=500, subsample=0.8;, score=0.970 total time=   7.4s
[CV 4/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=1, n_estimators=500, subsample=0.8;, score=0.969 total time=   8.0s
[CV 5/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=1, n_estimators=500, subsample=0.8;, score=0.972 total time=   7.2s
[CV 1/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=9, min_child_weight=1, n_estimators=500, subsam

In [ ]:
# Save the best model
with open('best_model.json', 'w') as f:
    json.dump(random_search.best_params_, f)

In [ ]:
random_search.best_params_

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'

In [ ]:
best_xgb = random_search.best_estimator_

# Perform cross-validation with the best model
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

fold_results = []

for fold, (train_index, val_index) in enumerate(cv.split(X, y), 1):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    best_xgb.fit(X_train, y_train)
    y_pred = best_xgb.predict(X_val)
    y_prob = best_xgb.predict_proba(X_val)
    
    # Calculate metrics
    accuracy, class_metrics, auc, f1, cm, avg_sensitivity, avg_specificity = calculate_metrics(y_val, y_pred, y_prob)
    
    # Calculate AUC for double dichotomy
    double_dichotomy_auc = calculate_double_dichotomy_auc(y_val, y_prob)
    
    # Log metrics for this fold
    metrics = {
        'fold': fold,
        'val_accuracy': accuracy,
        'val_auc': auc,
        'val_f1': f1,
        'avg_sensitivity': avg_sensitivity,
        'avg_specificity': avg_specificity,
        **double_dichotomy_auc,
        **{f'class_{class_names[i]}_sensitivity': metrics["sensitivity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{class_names[i]}_specificity': metrics["specificity"] for i, metrics in enumerate(class_metrics)},
        **{f'class_{class_names[i]}_f1': 2 * metrics["sensitivity"] * metrics["specificity"] / (metrics["sensitivity"] + metrics["specificity"]) for i, metrics in enumerate(class_metrics)}
    }
    
    custom_log(metrics, model_name='xgboost_optimized', log_dir='logs')
    
    # Plot confusion matrix for this fold
    plot_confusion_matrix(cm, class_names=class_names, epoch_num=0, model_name='xgboost_optimized', fold_num=fold)
    
    fold_results.append(metrics)

# Calculate and print average results across all folds
avg_results = {key: np.mean([fold[key] for fold in fold_results if key in fold]) 
               for key in fold_results[0].keys() if key != 'fold'}

print("Average results across all folds:")
for key, value in avg_results.items():
    print(f"{key}: {value}")

# Log average results
custom_log(avg_results, model_name='xgboost_optimized_average', log_dir='logs')